We saw how to solve a capacitated facility location problem (CFLP) in Julia using combined simulated annealing.

Here we look at a much more simpler approach in which we use a random walk to decide which facility to open and a simple heuristic to assign facilities to customer. 
In other words we look what would happen if we skip the inner layer algorithm in the combined simulated annealing.

But for the convenience of the reader we recall the capacitated facility location problem first.

## Problem statement

A company must select a subset of potential facility locations to minimize total cost associated with opening facilities and servicing customers. Each costumer has a specific demand and each facility has a fixed opening costs, a specific capacity and service costs based on the distance to customers.

## Mathematical model formulation

In order to have a more mathematical problem description we give a formulation as a mixed interger problem next:

### Sets

- $J=\{1,\ldots,m\}$ set of potential facilities
- $C=\{1,\ldots,n\}$ set of customers

### Parameters

- $c^f_j\in\mathbb{R}⁺$ fix opening cost of facility $j$
- $q_j\in\mathbb{N}^+$ capacity of facility $j$
- $d_i\in\mathbb{N}^+$ demand of customer $i$
- $c^v_{i,j}\in\mathbb{R}^+$ servicing costs from facility $j$ to customer $i$

### Decision variables

- $x_{ij}$, real, demand of customer $i$ served by facility $j$
- $y_j$, binary, 1 if and only if facility $j$ is open

### Objective

$$
\min \sum_j c^f_j \cdot y_j + \sum_{ij} c_{ij}x_{ij}
$$

### Constraints

- (c1) Each client's demand is served:

$$
\sum_j x_{ij} = d_i, \; \forall i \in C
$$

- (c2) A facility can serve a client only if its open:

$$
x_{ij} \leq d_i y_j, \forall i\in C, \forall j \in J
$$

- (c3) A facility can not provide more than its capacity

$$
\sum_i x_{ij} \leq q_j y_j, \forall j \in J
$$

- (c4) There is maximal number of facilities allowed to be opened

$$
\sum_j y_j \leq k, 
$$


### Problem formulation as a Mixed Interger Linear Programm


$$
\begin{array}{lll}
\min & \sum_j c^f_j \cdot y_j + \sum_{ij} c_{ij}x_{ij} &\\
s.t. & \sum_j x_{ij} = d_i & \forall i \in C\\
     & x_{ij} \leq d_i y_j & \forall i\in C, \forall j \in J\\
     & \sum_i x_{ij} \leq q_j y_j & \forall j \in J \\
     & \sum_j y_j \leq k & \\
     & x_{ij}\geq0 & \\
     & y_j\in\{0,1\}&
\end{array}
$$

In [1]:
using Random
using LinearAlgebra

In [2]:
# set seed for reporducibility
Random.seed!(1111);
Random.TaskLocalRNG();

## Implementation

Before we show an implementation of a simplified version of a simulated annealing for a CFLP and apply it to some examples, lets briefly give you a brief overview of the full implementation:

1. Data Structures 
2. I/O and Tests
3. Additonal functions e.g. to generate initial solutions
4. An heuristic asignment of client demand to open facilities
5. simmulated annealing implementation (only for the open facility descision)
8. Solving examples
9. Comparison

In [3]:
include("./SA_DataSturctures.jl")
include("./io.jl")
include("./tests.jl")
include("./SA_Algorithms.jl")

SA_client_facility_assignment (generic function with 5 methods)

## Heuristics 

Because the search space for the assignment variable $x_{ij}$ can become extremely large, we simply use a simple heuristic to compute a valid assignment of clients to facilites.
This is encoded in the function `client_facility_assignment` (see below).

The heuristic works as follows:

For each $i$ in customer do:
1. while demand[i] > 0
    find j in facility with free capacity such that the variable costs are minimal
    if demand[i] <= facility_capacity[j]
            assign[j,i] = demand[i]
            facility_capapacity[j] = facility_capacity[j] - demand[i]
            demand[i] = 0
        if demand[i] > facility_capacity[j]
            assgnm[j,i] = facility_capacity[j]
            demand[i] = demand[i] - facility_capacity[j]
            facility_capacity[j] = 0
   return assignment

In [4]:
function heuristic_client_facility_assignment(
        data::cflp_data, open_facilities::Vector{Bool})
    """
    returns m times n matrix with elements representing the demand of customer i served by facility j
    assignment is done by assigning the clients demand to the cheapest open facilitiy, more precisely:

    for i in customer
        while dem[i] > 0
            find j in facility with facility_cap[j] > 0 and cost_var[i,j] is minimal
        if dem[i] <= facility_cap[j]
            assignm[j,i] = dem[i]
            facility_cap[j] = facility_cap[j] - dem[i]
            dem[i] = 0
        if dem[i] > facility_cap[j]
            assgnm[j,i] = facility_cap[j]
            dem[i] = dem[i] - facility_cap[j]
            facility_cap[j] = 0
        return assignment    
    """

    # create assignment matrix
    assignment = zeros(Int, data.m, data.n)

    # get available facility capacities
    facility_cap = [data.capacity[i] * open_facilities[i] for i in 1:data.m] # usually elementwise multiplication in Julia is done by .*, but here we get a 3x3 matrix instead of a vector
    customer_demand = copy(data.demand)
    # initialize available facilities, because we want to use findmin later assign a big number to non open facilities
    facilities = [1. for j in 1:data.m]
    facilities[facility_cap .< 1] .= data.bigM_customer
    
    for i in 1:length(customer_demand)
        failsafe = 0
        while customer_demand[i] > 0 
            if failsafe > data.m * data.n
                break
            else 
                failsafe +=  1
            end
            j = findmin([data.cost_var[i,j] * facilities[j] for j in 1:data.m])[2]
            if customer_demand[i] <= facility_cap[j]
                assignment[j,i] += customer_demand[i]
                facility_cap[j] -= customer_demand[i]
                customer_demand[i] = 0
                facilities[facility_cap .< 1] .= data.bigM_customer
            end
            if customer_demand[i] > facility_cap[j]
                assignment[j,i] += facility_cap[j]
                customer_demand[i] -= facility_cap[j]
                facility_cap[j] = 0
                facilities[facility_cap .< 1] .= data.bigM_customer
            end
            
            # if all demand is allocated stop loop
            if sum(customer_demand) == 0
                break
            end
        end
    end
    return assignment
end

heuristic_client_facility_assignment (generic function with 1 method)

Our simulated annealing with this heuristic can be implemented as follows:

In [5]:
function SA_cflp_with_heuristic(
        data::cflp_data, k::Int64, start_temp::Float64 = 10.0, end_temp::Float64 = 1.0, alpha::Float64 = 0.5, max_iter::Int64 = 100)
    """
    TBA
    """
    # generate initial solution 
    current_solution = generate_initial_solution(data, k)
    best_solution = current_solution

    # iterate
    temp = start_temp
    while temp > end_temp
        for _ in 1:max_iter
            # generate a new decision on open facilities
            candidate_solution = generate_candidate_facility(data, current_solution, k)
            
            if  test_cflp_solution(data, candidate_solution, k) && candidate_solution.cost < best_solution.cost 
                best_solution = candidate_solution
            end

            if candidate_solution.cost < current_solution.cost
                current_solution = candidate_solution
            else
                acceptance_prob = exp( (current_solution.cost - candidate_solution.cost) / temp )
                if rand() < acceptance_prob
                    current_solution = candidate_solution
                end
            end
            
        end
        
        temp *= alpha
    end

    if !test_cflp_solution(data, best_solution,k)
        println("did not found a feasible solution")
    end
    
    return  best_solution
end

SA_cflp_with_heuristic (generic function with 5 methods)

## Examples

### A simple CFLP

The following problem data for a CFLP is taken from [Mathematical Optimization: Solving Problems using SCIP and Python](https://scipbook.readthedocs.io/en/latest/flp.html).

In [6]:
# data
demand = [80, 270, 250, 160, 180]
capacity = [500,500,500]
cost_fix = [1000.,1000.,1000.]
cost_var = [4. 6. 9.; 5. 4. 7.; 6. 3. 4.; 8. 5. 3.; 10. 8. 4.]

data = cflp_data(cost_fix, cost_var, capacity, demand)
sol = SA_cflp_with_heuristic(data, 2)
print_solution(sol)

Total cost are:5610.0
Open facilities:[2, 3]
Assignment:
customer   1 gets       80.000000 from facility   2
customer   2 gets      270.000000 from facility   2
customer   3 gets      150.000000 from facility   2
customer   3 gets      100.000000 from facility   3
customer   4 gets      160.000000 from facility   3
customer   5 gets      180.000000 from facility   3


We recall that the optimal solution was

    Total cost are:5370.000000000006
    Open facilities:[2, 3]


and a solution of the combined simulated annealing was
    
    Total cost are:5610.0
    Open facilities:[2, 3]

### Example 2

In [7]:
using JLD2

d = load("../data/data.jld2")["data"]
dont_print_all = true
data = cflp_data(d["cost_fix"], d["cost_var"], d["capacity"], d["demand"])

k = 9
example2 = SA_cflp_with_heuristic(data, k)
print_solution(example2, dont_print_all)

Total cost are:14704.919999999998
Open facilities:[1, 3, 5, 7, 8, 9, 10, 11, 12]
Assignment:
customer   1 gets       18.000000 from facility   9
customer   2 gets       21.000000 from facility   7
customer   3 gets       12.000000 from facility  11
customer   4 gets       14.000000 from facility   3
customer   5 gets       18.000000 from facility   3


We recall the MILP solution was 

    Total cost are:14277.130000000001
    Open facilities:[1, 3, 5, 6, 8, 9, 10, 11, 13]

and a solution of the combined simulated annealing was

    Total cost are:14278.269999999999
    Open facilities:[1, 3, 5, 6, 8, 9, 10, 11, 13]